# TUMOR ground station data display

This code is a part of the ground station segment for [TEK5720](https://www.uio.no/studier/emner/matnat/its/TEK5720/) TUMOR (Targeting Ultraviolet Mid-Ozone layer Radiation) mission

Group Blip-Blop:
* Joachim Thomle Karlsen
* Tobias Mellum
* Michał Jan Odorczuk
* Vytenis Orlauskis
* Yawar Seraj

Code made by Michał Jan Odorczuk

This code creates an html file with the GPS track of the probe based on the data retreived from the ground station module

Ground station module code is available [here](https://github.com/VytenisO/tumor/blob/main/flightVersion/groundStation.ino)

Ground station receives data transmitted by the [CanSat module](https://github.com/VytenisO/tumor/blob/main/flightVersion/cansat.ino)

Ground station data is saved to a file with the [data capture script](https://github.com/VytenisO/tumor/blob/main/flightVersion/data_capture.ipynb)

## imports

Please install [numpy](https://pypi.org/project/numpy/), [folium](https://pypi.org/project/folium/) and [pandas](https://pypi.org/project/pandas/)

In [ ]:
import pandas as pd
import folium
from time import sleep
from os import path, listdir
import numpy as np

## Initial data

`x`, `y` are the focus coordinates in degrees. Please change for other initial focus coordinates.

`zoom` is the initial zoom in the tracker.

`lon_low` and `lon_high` are respectively lower and upper limits of longitudinal values. All the points with longitudes outside of those will be skipped in the tracker.

`lat_low` and `lat_high` are respectively lower and upper limits of latitudinal values. All the points with latitudes outside of those will be skipped in the tracker.

`mapfile` is the name of the html file containing the tracker data.

In [ ]:
x, y = 69.2965, 16.032
zoom = 13

lon_low = 5
lon_high = 30
lat_low = 60
lat_high = 80

m = folium.Map(location=[x, y], zoom_start=zoom)
marker = folium.Marker([x, y])
marker.add_to(m)
mapfile = 'gps_map.html'

In [ ]:
file_times = [(f, path.getmtime(f)) for f in listdir('.') if path.isfile(f) and f[-3:] == 'csv']

sorted_files = sorted(file_times, key=lambda x: x[1], reverse=True)
data_source = sorted_files[0][0]

while True:
    df = pd.read_csv(data_source, comment='#')
    lons = (pd.to_numeric(df['lon_deg']) + pd.to_numeric(df['lon_min']) / 60 + pd.to_numeric(df['lon_sec']) / 3600).to_numpy()
    lats = (pd.to_numeric(df['lat_deg']) + pd.to_numeric(df['lat_min']) / 60 + pd.to_numeric(df['lat_sec']) / 3600).to_numpy()
    mask = (lons<lon_high)&(lons>lon_low)&(lats<lat_high)&(lats>lat_low)
    lons = lons[mask]
    lats = lats[mask]
    trajectory = np.array([lats, lons]).T
    marker.location = list(trajectory[-1,:])
    trajectory_line = folium.PolyLine(locations=list(trajectory), color='blue')
    m.add_child(trajectory_line)
    m.save(mapfile)
    sleep(5)